<a href="https://colab.research.google.com/github/manish996330/Predicting-Future-Exchange-Rates-with-Linear-Regression/blob/main/Predict_Prices_using_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting the Future Exchange Rates by training a Linear Regression Model using the historical data.

## Loading the required libraries and packages

Now, let's load all the required packages to the project file, so that we can efficiently code, without having to bother about calling the packages while coding.

This code imports the necessary libraries for data analysis and visualization such as pandas, numpy, plotly.graph_objs and sklearn.linear_model. It also imports the mean_absolute_percentage_error function from sklearn.metrics and the drive function from google.colab.

The LinearRegression function from sklearn.linear_model is used to perform linear regression on the data. The mean_absolute_percentage_error function from sklearn.metrics is used to calculate the mean absolute percentage error of the model.

The drive function from google.colab is used to mount Google Drive to the Colab notebook.

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from google.colab import drive

## Connect to Drive and load the historical data of the Currency Pair (EUR - USD)

Now, lets mount the Google Drive for the project file, so that the downloaded historical data of a specific ticker can be accessed, on which our model can be trained.

In [2]:
drive.mount ('/content/drive')
project_directory = '/content/drive/MyDrive/St Clair/AI Project (DAB106)'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


This code reads a CSV file named EURUSD13114.csv located in the project_directory and stores it in a pandas DataFrame called df. The Date column is converted to a datetime format using the pd.to_datetime function. The set_index function is used to set the Date column as the index of the DataFrame.

In [3]:
df = pd.read_csv(project_directory+'/EURUSD23DAY.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

## Data Cleaning and Pre - Processing

Now, from the raw data pull, we can see that this data, in its current state, cannot be used for training the model. Hence, we need to clean the data and extract the target variable and the features of the dataset that are contributing to the value of the target variable.

This code creates a variable X which is a 2D numpy array containing the Open, High, Low, and Close prices of the df DataFrame, except for the last row. The y variable is also a 2D numpy array containing the Open, High, Low, and Close prices of the df DataFrame, except for the first row. The X variable is used as the input features to predict the y variable.

In [4]:
X = df[['Open', 'High', 'Low', 'Close']].values[:-1]  # Use open, high, low, and close prices as features
y = df[['Open', 'High', 'Low', 'Close']].values[1:]   # Predict the next period's open, high, low, and close prices

### Splitting the data into traning and validation datasets.

This code creates a variable train_size which is 90% of the data of X. The X_train and y_train variables are created by slicing the X and y variables up to the train_size index. The X_test and y_test variables are created by slicing the X and y variables from the train_size index to the end of the array.

In [5]:
train_size = int(0.9 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

## Building the Model to train the data

This code creates a LinearRegression object called lr. The n_epochs variable is set to 100. The lr.fit function is used to fit the model to the training data. The lr.predict function is used to make predictions on the testing data. The lr.score function is used to calculate the accuracy of the model.

A DataFrame called df_pred is created with the predicted Open, High, Low, and Close prices and corresponding dates. The set_index function is used to set the Date column as the index of the DataFrame. The last_date variable is set to the last date in the df DataFrame. The next_day variable is set to the predicted price for the next day. The predicted price for the next day is added to the df_pred DataFrame. The pd.concat function is used to combine the actual and predicted dataframes.

The accuracy of the model is printed at the end of the code.

In [6]:
lr = LinearRegression()
n_epochs = 100
for epoch in range(n_epochs):
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_test)

    # Calculate the accuracy of the model
    accuracy = r2_score(y_test, y_pred)

    # Create a dataframe with the predicted open, high, low, and close prices and corresponding dates
    df_pred = pd.DataFrame({'Date': df.index.values[train_size+1:], 'Open': y_pred[:, 0], 'High': y_pred[:, 1], 'Low': y_pred[:, 2], 'Close': y_pred[:, 3]})
    df_pred.set_index('Date', inplace=True)

    # Predict the next day
    last_date = df.index[-1]
    next_day = lr.predict(np.array([df[['Open', 'High', 'Low', 'Close']].values[-1]]).reshape(1, -1))

    # Add predicted price for the next day to the dataframe
    last_date = last_date + pd.DateOffset(days=1)
    df_pred.loc[last_date] = next_day[0]

    # Combine actual and predicted dataframes
    df_combined = pd.concat([df, df_pred])

print(accuracy)

0.9965722089444651


This code calculates the accuracy of predictions made by a machine learning model for predicting stock prices. 

1. The code calculates the absolute difference between the predicted and actual open prices by subtracting the predicted open prices (df_pred['Open']) from the actual open prices (df['Open']) and taking the absolute value of the result using the abs method. This is stored in the pip_diff_open variable.

2. The code then calculates the percentage accuracy of the open price predictions by dividing the absolute difference (pip_diff_open) by the actual open prices (df['Open']) and multiplying by 100. This is stored in the accuracy_open variable.

3. The code calculates the average deviation across all open price predictions by taking the mean of the accuracy_open variable using the mean method. This is stored in the average_accuracy_ variable.

4. The code prints the average deviation of the predicted open prices from the actual open prices using a formatted string.

5. The same process is repeated for high, low, and close prices, with the results being stored in pip_diff_high, accuracy_high, average_accuracy_high, pip_diff_low, accuracy_low, average_accuracy_low, pip_diff_close, accuracy_close, and average_accuracy_close variables respectively.

Overall, this code provides a way to evaluate the deviation of a model’s predictions for open, high, low, and close prices in a financial market by calculating the average deviation of the predicted prices from the actual prices.

In [7]:
# ("Deviation of Predicted Open from Actual Open Prices")
pip_diff_open = (df['Open'] - df_pred['Open'])
# Set negative values to positive
pip_diff_open = pip_diff_open.abs()
# Calculate the percentage accuracy of the predictions
accuracy_open = (abs(pip_diff_open) / df['Open'])
# Calculate the average accuracy across all predictions
average_accuracy_open = accuracy_open.mean()
print(f"Average Deviation of Predicted Open from Actual Open Prices is {(average_accuracy_open * 10000):.5f}%")

# ("Deviation of Predicted High from Actual High Prices")
pip_diff_high = (df['High'] - df_pred['High'])
# Set negative values to positive
pip_diff_high = pip_diff_high.abs()
# Calculate the percentage accuracy of the predictions
accuracy_high = (abs(pip_diff_high) / df['High'])
# Calculate the average accuracy across all predictions
average_accuracy_high = accuracy_high.mean()
print(f"Average Deviation of Predicted High from Actual High Prices is {average_accuracy_high * 10000:.5f}%")

# ("Deviation of Predicted Low from Actual Low Prices")
pip_diff_low = (df['Low'] - df_pred['Low'])
# Set negative values to positive
pip_diff_low = pip_diff_low.abs()
# Calculate the percentage accuracy of the predictions
accuracy_low = (abs(pip_diff_low) / df['Low'])
# Calculate the average accuracy across all predictions
average_accuracy_low = accuracy_low.mean()
print(f"Average Deviation of Predicted Low from Actual Low Prices is {average_accuracy_low * 10000:.5f}%")


# ("Deviation of Predicted Close from Actual Close Prices")
pip_diff_close = (df['Close'] - df_pred['Close'])
# Set negative values to positive
pip_diff_close = pip_diff_close.abs()
# Calculate the percentage accuracy of the predictions
accuracy_close = (abs(pip_diff_close) / df['Close'])
# Calculate the average accuracy across all predictions
average_accuracy_close = accuracy_close.mean()
print(f"Average Deviation of Predicted Close from Actual Close Prices is {average_accuracy_close * 10000:.5f}%")


# Add the accuracy columns to the df_pred dataframe and round to 2 decimal places
df_pred['Accuracy High'] = accuracy_high.round(5)
df_pred['Accuracy Low'] = accuracy_low.round(5)
df_pred["Accuracy Open"] = accuracy_open.round(5)
df_pred["Accuracy Close"] = accuracy_close.round(5)

Average Deviation of Predicted Open from Actual Open Prices is 1.85297%
Average Deviation of Predicted High from Actual High Prices is 23.75523%
Average Deviation of Predicted Low from Actual Low Prices is 24.03471%
Average Deviation of Predicted Close from Actual Close Prices is 29.45687%


As we can see above, the average deviation among the predicted values in much less in the "Open" variable at only 1.853%, whereas that of the "Close" variable is on the high side at 29.457%

## Plotting the Predictions of the model

This code creates a plot of actual and predicted prices over time for a given stock. It uses the Plotly library to create a candlestick chart of the actual prices and two line charts of the predicted high and low prices. The chart also includes annotations that show the deviation of each predicted price from the actual price as a percentage.

The first three lines of code create the traces for the actual and predicted prices. The go.Candlestick function is used to create a candlestick chart of the actual prices, while go.Scatter is used to create line charts of the predicted high and low prices.

The next four lines combine the traces into a single list called data and set the layout for the chart. The layout includes a title, hides the x-axis range slider, and sets the orientation and position of the legend.

The next four lines add annotations to the chart that show the deviation of each predicted price from the actual price as a percentage. These annotations are positioned above each respective trace using x- and y-coordinates.

Finally, fig.show() is called to display the chart in an interactive window.

In [8]:
# Create the actual and predicted price traces
trace_actual = go.Candlestick(x=df.index, open=df['Open'], high=df['High'], low=df['Low'], close=df['Close'], name='Actual Prices', increasing_line_color='green', decreasing_line_color='red', opacity=0.4)
trace_predicted_low = go.Scatter(x=df_pred.index, y=df_pred['Low'], name='Predicted Day Low', mode='lines', line=dict(color='darkred'), opacity=0.7, text=df_pred['Accuracy Low'].round(2))
trace_predicted_high = go.Scatter(x=df_pred.index, y=df_pred['High'], name='Predicted Day High', mode='lines', line=dict(color='darkgreen'), opacity=0.7, text=df_pred['Accuracy High'].round(2))

# Combine the traces and set the layout
data = [trace_actual, trace_predicted_low, trace_predicted_high]
layout = go.Layout(title='Actual and Predicted Prices over Time', xaxis_rangeslider_visible=False, legend=dict(orientation='h', y=1.1))

# Create the figure and show the plot
fig = go.Figure(data=data, layout=layout)

# Add annotations for average_accuracy_low and average_accuracy_high
fig.add_annotation(x=-0.1, y=df['High'].max()*1.20,
                   text=f"Deviation of Predicted Open: {average_accuracy_open:.2f}%",
                   showarrow=False, font=dict(size=14))
fig.add_annotation(x=-0.1, y=df['High'].max()*1.15,
                   text=f"Deviation of Predicted High: {average_accuracy_high:.2f}%",
                   showarrow=False, font=dict(size=14))
fig.add_annotation(x=-0.1, y=df['Low'].max()*1.10,
                   text=f"Deviation of Predicted Low: {average_accuracy_low:.2f}%",
                   showarrow=False, font=dict(size=14))
fig.add_annotation(x=-0.1, y=df['Low'].max()*1.05,
                   text=f"Deviation of Predicted Close: {average_accuracy_close:.2f}%",
                   showarrow=False, font=dict(size=14))
fig.show()

As you can see above, we have started out with the data points from the 1970s all the way to 2023. We have plotted the predicted day "High" and "Low", so that we can predict the day range and then compare it with the actual price. We can see that the model managed to predict the ranges more or less accurately.

The fallacy here is that, currency pairs are generally influenced by real time events, and are pricing is not completely mathematical. Hence, we can see the deviation from the prediction at some points.

In [9]:
# Create the actual and predicted price traces
trace_deviation_high = go.Scatter(x=df_pred.index, y=df_pred['Accuracy High'], name='Deviation of High Prices', mode='lines', line=dict(color='darkgreen'), opacity=0.7, text=df_pred['Accuracy High'].round(5))
trace_deviation_low = go.Scatter(x=df_pred.index, y=df_pred['Accuracy Low'], name='Deviation of Low Prices', mode='lines', line=dict(color='darkred'), opacity=0.7, text=df_pred['Accuracy Low'].round(5))

# Combine the traces and set the layout
data = [trace_deviation_high, trace_deviation_low]
layout = go.Layout(title='Deviation of High and Low Predictions over Time', xaxis_rangeslider_visible=False, legend=dict(orientation='h', y=1.1), yaxis=dict(tickformat='.2%'))

# Create the figure and show the plot
fig = go.Figure(data=data, layout=layout)

fig.show()

And the above visualisation shows the variation of deviation of predicted High and Low from the actual values over time.

## Return the Prediction for the next calender date of the data set.

Now, from the above predicted data, lets return the last entry from the dataset, which would be the predicted prices for the next calender data, of the last data from the actual training and validation data set. Lets also return the previous actual prices so as to have a frame of reference for comparision.

In [10]:
print("Predicted Prices for the Next Calender Date which is " + str(df_pred.tail(1).index[0]))
print("Our prediction deduces that the day high for the above date would be " + str(df_pred["High"].tail(1).T[0]) + " and the day low would be " + str(df_pred["Low"].tail(1).T[0]))

Predicted Prices for the Next Calender Date which is 2023-03-24 00:00:00
Our prediction deduces that the day high for the above date would be 1.0869911666229513 and the day low would be 1.0780897607297009


From the above, we can see that our model predicts the currency pair EUR - USD will trade in the range of 1.086991 and 1.07809, if there are no macro events influencing the price during trading hours. However, it is important to factor in the average deviation in values so that the user can be prepared to handle the deviation in their actual trading setup.
